In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

/var/folders/4r/8l8ztg457hq357p0kylhwhfw0000gn/T/ipykernel_35622/3632437423.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [23]:
from linearmodels.panel import PanelOLS

sc_employment = pd.read_csv(
    "./data/processed_employment/Santa_Clara_County,_California.csv"
)
sc_real_estate = pd.read_csv("./data/processed_zillow/Santa_Clara_County.csv")

data = sc_real_estate
data = pd.concat([data, sc_employment], axis=1)
data = data.dropna(subset=["Year-Month"])
data["county"] = "Santa Clara"
data["constant"] = 1
data["Date"] = pd.to_datetime(data["Date"])
data["Year"] = data["Date"].dt.year
data["Quarter"] = data["Date"].dt.quarter
data["Year_Quarter"] = data["Year"].astype(str) + "Q" + data["Quarter"].astype(str)
year_quarter_dummies = pd.get_dummies(
    data["Year_Quarter"], prefix="YQ", drop_first=True
)
data = pd.concat([data, year_quarter_dummies], axis=1)
print(year_quarter_dummies)
print(data.tail(10))

data = data.set_index(["county", "Date"])

mod = PanelOLS(data.Price, data[["Employment"] + year_quarter_dummies.columns.tolist() + ["constant"]])
res = mod.fit()
print(res)


# # Assume your data includes a 'date' column in datetime format, from which we can extract quarters
# data["quarter"] = data["Date"].dt.to_period("Q")  # Create quarter periods

# # Convert 'quarter' to dummy variables (time fixed effects) and merge back into the main dataframe
# quarter_dummies = pd.get_dummies(data["quarter"], prefix="quarter", drop_first=True)
# data = pd.concat([data, quarter_dummies], axis=1)

# # print(data)
# # # # Set the data to a panel data format with a MultiIndex
# data = data.set_index(["county"])
# print(data)
# # Define the formula for PanelOLS
# # Add all time dummy variables to the formula as fixed effects, plus the main variables
# time_fixed_effects = " + ".join(quarter_dummies.columns)
# formula = f"Price ~ Employment + {time_fixed_effects}"

# # Fit the fixed effects model with both entity (county) and time fixed effects
# model = PanelOLS.from_formula(formula, data=data)
# fe_results = model.fit()

# print(fe_results)

     YQ_2011Q2  YQ_2011Q3  YQ_2011Q4  YQ_2012Q1  YQ_2012Q2  YQ_2012Q3  \
0        False      False      False      False      False      False   
1        False      False      False      False      False      False   
2        False      False      False      False      False      False   
3         True      False      False      False      False      False   
4         True      False      False      False      False      False   
..         ...        ...        ...        ...        ...        ...   
154      False      False      False      False      False      False   
155      False      False      False      False      False      False   
156      False      False      False      False      False      False   
157      False      False      False      False      False      False   
158      False      False      False      False      False      False   

     YQ_2012Q4  YQ_2013Q1  YQ_2013Q2  YQ_2013Q3  ...  YQ_2021Q4  YQ_2022Q1  \
0        False      False      False      Fal